In [9]:
import torch.nn as nn
import math
import torch
import torch.nn.functional as F

def conv3x3(in_planes, out_planes, stride=1, groups=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, groups=groups, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)
    
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)
    
class Rep_SEBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None,baseWidth = 7,scale = 4):
        super(Rep_SEBasicBlock, self).__init__()
        #width = int(math.floor(planes * (baseWidth/16.0)))
        width = int(math.floor(planes/scale))

        self.conv1 = conv3x3(inplanes, width * scale, stride)
        self.bn1 = nn.BatchNorm2d(width*scale)
        
        if downsample is None:
            self.conv_rep = nn.Conv2d(inplanes,inplanes,kernel_size=1)
            self.bn_rep = nn.BatchNorm2d(inplanes)
        
        if scale == 1:
            self.nums = 1
        else:
            self.nums = scale -1
        convs = []
        bns = []
        for i in range(self.nums):
            convs.append(conv3x3(width,width))
            bns.append(nn.BatchNorm2d(width))
        self.convs = nn.ModuleList(convs)
        self.bns = nn.ModuleList(bns)
        self.relu = nn.ReLU(inplace=True)

        self.downsample = downsample
        self.stride = stride
        self.width = width
        self.scale = scale
        self.se = SELayer(planes, 16)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        spx = torch.split(out, self.width, 1)
        
        for i in range(self.nums):
            if i==0:
                sp = spx[i]
            else:
                sp = sp + spx[i]
            sp = self.convs[i](sp)
            sp = self.relu(self.bns[i](sp))
            if i==0:
                out = sp
            else:
                out = torch.cat((out,sp),1)
        if self.scale != 1:
            out = torch.cat((out,spx[self.nums]),1)
        
        out = self.se(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        else:
            rep = self.relu(self.bn_rep(self.conv_rep(residual)))
            out += rep

        out += residual
        out = self.relu(out)

        return out




class Res2Net(nn.Module):

    def __init__(self, block, layers, in_planes=32, baseWidth = 26, scale = 4, num_classes=1000):
        self.inplanes = in_planes
        super(Res2Net, self).__init__()
        self.baseWidth = baseWidth
        self.scale = scale
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=3, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, in_planes, layers[0])
        self.layer2 = self._make_layer(block, in_planes*2, layers[1], stride=2)
        self.layer3 = self._make_layer(block, in_planes*4, layers[2], stride=2)
        self.layer4 = self._make_layer(block, in_planes*8, layers[3], stride=2)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample=downsample, baseWidth = self.baseWidth, scale=self.scale))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, baseWidth = self.baseWidth, scale=self.scale))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        #x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        return x

def Res2Net34SE_Rep(in_planes=32, baseWidth = 26, scale = 8, **kwargs):
    """Constructs a Res2Net-50 model.
    Res2Net-50 refers to the Res2Net-50_26w_4s.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = Res2Net(Rep_SEBasicBlock, [3, 4, 6, 3], in_planes=in_planes, baseWidth = baseWidth, scale = scale, **kwargs)

    return model


In [10]:
model = Res2Net34SE(in_planes=32, baseWidth = 26, scale = 8)

In [11]:
x = torch.rand(4,1,80,100)

In [12]:
y = model(x)

In [50]:
y.shape

torch.Size([4, 256, 6, 7])

In [13]:
conv1 = nn.Conv2d(32,32,kernel_size=1)

In [14]:
x = torch.rand(4,32,80,100)
y = conv1(x)

In [19]:
bn2 = nn.BatchNorm2d(80)

In [20]:
bn2(y).shape

RuntimeError: running_mean should contain 32 elements not 80